## Дерево решений

Задание
1. Там, где написано "Ваш код", нужно реализовать метод или часть метода
2. Там, где написано "Что делает этот блок кода?", нужно разобраться в блоке кода и в комментарии написать, что он делает
3. Добиться, чтобы в пункте "Проверка скорости работы" Ваша реализация работала чуть быстрее, чем у дерева из sklearn (это возможно, так как мы реализуем только малую часть функциональности)
4. Добиться, чтобы в пункте "Проверка качества работы" Ваша реализация работала так же или качественнее, чем у дерева из sklearn
5. Применить реализованное дерево решений для задачи Titanic на kaggle. Применить для той же задачи дерево решений из sklearn. Применить кросс-валидацию для подбора параметров. Сравнить с результатами предыдущих моделей. Если результат улучшился - сделать сабмит. Написать отчет о результатах.

In [1]:
from time import time

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from scipy import optimize
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeClassifier

%matplotlib inline

# Весь код, который нужно разобрать, будем дебажить

In [2]:
x=np.array([[3,2],[4,4],[2,4]])

In [3]:
y=np.array([[1], [2], [2]])

In [4]:
sorted_idx = x.argsort()
sorted_idx

array([[1, 0],
       [0, 1],
       [0, 1]], dtype=int64)

сортируем внутри строк

In [5]:
sorted_x, sorted_y = x[sorted_idx], y[sorted_idx]
sorted_x, sorted_y 

(array([[[4, 4],
         [3, 2]],
 
        [[3, 2],
         [4, 4]],
 
        [[3, 2],
         [4, 4]]]), array([[[2],
         [1]],
 
        [[1],
         [2]],
 
        [[1],
         [2]]]))

пока непонятно что, скорее всего передаем ОДИН признак. Попробуем еще раз

In [6]:
x=np.array([7, 6, 5,4,3,2,1,0])
y=np.array([1, 1, 1,1,0,0,2,2])

In [7]:
sorted_idx = x.argsort()
sorted_idx

array([7, 6, 5, 4, 3, 2, 1, 0], dtype=int64)

In [8]:
sorted_x, sorted_y = x[sorted_idx], y[sorted_idx]
sorted_x, sorted_y 

(array([0, 1, 2, 3, 4, 5, 6, 7]), array([2, 2, 0, 0, 1, 1, 1, 1]))

Отсортировали значения по возрастанию признака

In [9]:
class_number = np.unique(y).shape[0]
class_number

3

In [11]:
np.unique(y)

array([0, 1, 2])

Получили количество уникальный классов

In [14]:
splitted_sorted_y = sorted_y[1:-1]
splitted_sorted_y

array([2, 0, 0, 1, 1, 1])

пока сделаем так, что индексы берут средний список

In [15]:
splitted_sorted_y = sorted_y[1:-1]
r_border_ids = np.where(splitted_sorted_y[:-1] != splitted_sorted_y[1:])[0] + (1 + 1)

r_border_ids

array([2, 4], dtype=int64)

получили список, где будем делить (правые границы изменения классов)

Если класс по размеру не разделим, то возвращаем float('+inf'), None

In [16]:
one_hot_code = np.zeros((r_border_ids.shape[0], class_number))
one_hot_code

array([[0., 0., 0.],
       [0., 0., 0.]])

Кодируем классы - количество строк - количество возможных делений, количество колоннок - количество классов

In [17]:
eq_el_count = r_border_ids - np.append([1], r_border_ids[:-1])
eq_el_count

array([1, 2], dtype=int64)

количество одинаковых элементов (после отсечения)

In [18]:
np.arange(r_border_ids.shape[0])

array([0, 1])

получаем индексы для всех строк

In [19]:
sorted_y[r_border_ids - 1]

array([2, 0])

получаем значения левых границ

классы должны быть закодированы, начиная с 0

In [20]:
one_hot_code[np.arange(r_border_ids.shape[0]), sorted_y[r_border_ids - 1]] = 1
one_hot_code

array([[0., 0., 1.],
       [1., 0., 0.]])

In [21]:
 eq_el_count.reshape(-1, 1)

array([[1],
       [2]], dtype=int64)

In [22]:
class_increments = one_hot_code * eq_el_count.reshape(-1, 1)
class_increments        

array([[0., 0., 1.],
       [2., 0., 0.]])

In [23]:
np.bincount(sorted_y[:1], minlength=class_number)

array([0, 0, 1], dtype=int64)

In [24]:
class_increments[0] = class_increments[0] + np.bincount(sorted_y[:1], minlength=class_number)
class_increments

array([[0., 0., 2.],
       [2., 0., 0.]])

Получили количество классов КРОМЕ находящиегося справа на ВСЕЙ выборке

In [25]:
l_class_count = np.cumsum(class_increments, axis=0)        
l_class_count

array([[0., 0., 2.],
       [2., 0., 2.]])

наконец-то получили количество классов слева при разделении

In [26]:
r_class_count = np.bincount(y) - l_class_count
r_class_count

array([[2., 4., 0.],
       [0., 4., 0.]])

получили количество классов справа

In [27]:
l_sizes = r_border_ids.reshape(l_class_count.shape[0], 1)
l_sizes

array([[2],
       [4]], dtype=int64)

сумма классов слева

In [28]:
r_sizes = sorted_y.shape[0] - l_sizes
r_sizes

array([[6],
       [4]], dtype=int64)

сумма классов справа

С кодом разобрались. Теперь тестируем критерии информативности.
Так как берется левая и правая часть и берется минимум, то просто берем сумму левой и правой части по информативности

In [50]:
    def __gini(l_c, l_s, r_c, r_s):
        #l_class_count, l_sizes, r_class_count, r_sizes
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')
        
        sum_l = np.sum(np.power(l_c, 2), axis=1)
        c_l = np.power(l_s.reshape([1, 2])[0], 2)
        
        sum_r = np.sum(np.power(r_c, 2), axis=1)
        c_r = np.power(r_s.reshape([1, 2])[0], 2)
        return 2 - sum_l/c_l - sum_r/c_r

In [52]:
__gini(l_class_count, l_sizes, r_class_count, r_sizes)

array([0.44444444, 0.5       ])

In [53]:
l_class_count, l_sizes, r_class_count, r_sizes

(array([[0., 0., 2.],
        [2., 0., 2.]]), array([[2],
        [4]], dtype=int64), array([[2., 4., 0.],
        [0., 4., 0.]]), array([[6],
        [4]], dtype=int64))

Проверим

In [57]:
1-4/4+1-4/36-16/36, 1-8/16+1-16/16

(0.4444444444444444, 0.5)

Совпало. Идем дальше

In [58]:
l_class_count/l_sizes

array([[0. , 0. , 1. ],
       [0.5, 0. , 0.5]])

In [64]:
def __entropy(l_c, l_s, r_c, r_s):
    p_l = l_c/l_s
    p_r = r_c/r_s
    p_ll = np.log2(p_l, where = p_l!=0)
    p_rl = np.log2(p_r,  where = p_r!=0)
    return - np.sum(p_l*p_ll, axis=1) - np.sum(p_r*p_rl, axis=1) 

In [67]:
__entropy(l_class_count, l_sizes, r_class_count, r_sizes)

array([0.91829583, 1.        ])

In [68]:
def __misclass(l_c, l_s, r_c, r_s):
    p_l = l_c/l_s
    p_r = r_c/r_s
    m_l = np.max(p_l, axis=1)
    m_r = np.max(p_r, axis=1)
    return 2 - m_l - m_r

In [72]:
__misclass(l_class_count, l_sizes, r_class_count, r_sizes)

array([0.33333333, 0.5       ])

In [73]:
import math

Теперь определения индексов feature

In [75]:
def __get_feature_ids_sqrt(n_feature):
    feature_ids = np.arange(n_feature)
    np.random.shuffle(feature_ids)
    return feature_ids[:int(n_feature**0.5)]
        
def __get_feature_ids_log2(n_feature):
    feature_ids = np.arange(n_feature)
    np.random.shuffle(feature_ids)
    return feature_ids[:int(math.log2(n_feature))]

def __get_feature_ids_N(n_feature):
    return np.arange(n_feature)

In [77]:
__get_feature_ids_sqrt(4)

array([1, 2])

In [79]:
__get_feature_ids_log2(8)

array([7, 3, 4])

In [81]:
__get_feature_ids_N(3)

array([0, 1, 2])

все кажется нормально

Дальше идут куски тестирования итеративного алгоритма

In [92]:
ar = np.array([1,3,4,3,3,0])

In [93]:
np.max(np.unique(ar, return_counts=True)[1])/ar.shape[0]

0.5

In [94]:
clas, count = np.unique(ar, return_counts=True)

In [95]:
clas, count

(array([0, 1, 3, 4]), array([1, 1, 3, 1], dtype=int64))

In [97]:
np.argmax(count)

2

In [98]:
clas[np.argmax(count)]

3

In [ ]:
class MyDecisionTreeClassifier:
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1

    def __init__(self, min_samples_split=2, max_depth=None, 
                 sufficient_share=1.0, criterion='gini', max_features=None):
        #комментарии для себя:
        #ожидается в dict:
        #ключ - id, как при хранении дерева в массиве, так:
        # 0 - слева 1, справа 2
        # 1 - слева 2*1+1=3, справа 2*1+2=4
        # 2 - слева 2*2+1=5, справа 2*2+2 = 6
        # В самом объекте ожидается массив из 3 элементов, 
        # первый - код LEAF_TYPE, промежуточные NON_LEAF_TYPE, конечные - LEAF_TYPE
        # для NON_LEAF_TYPE - 2 и 3 - feature_id, threshold
        # для LEAF_TYPE - класс и вероятность
        self.tree = dict()
        
        # минимальное количество элементов в листе для разделения
        self.min_samples_split = min_samples_split
        #максимальная глубина
        self.max_depth = max_depth
        #доля преобладающего класса
        self.sufficient_share = sufficient_share
        self.num_class = -1
        #функция критерии
        if criterion == 'gini':
            self.G_function = self.__gini
        elif criterion == 'entropy':
            self.G_function = self.__entropy
        elif criterion == 'misclass':
            self.G_function = self.__misclass
        else:
            print 'invalid criterion name'
            raise

        #количество используемых призаков
        if max_features == 'sqrt':
            self.get_feature_ids = self.__get_feature_ids_sqrt
        elif max_features == 'log2':
            self.get_feature_ids = self.__get_feature_ids_log2
        elif max_features == None:
            self.get_feature_ids = self.__get_feature_ids_N
        else:
            print 'invalid max_features name'
            raise

    def __gini(self, l_c, l_s, r_c, r_s):
        #l_class_count, l_sizes, r_class_count, r_sizes
        #См. выше в отладке
        return __gini(l_c, l_s, r_c, r_s)
    
    def __entropy(self, l_c, l_s, r_c, r_s):
        #См. выше в отладке
        return __entropy(l_c, l_s, r_c, r_s)

    def __misclass(self, l_c, l_s, r_c, r_s):
        #См. выше в отладке
        return __misclass(l_c, l_s, r_c, r_s)

    def __get_feature_ids_sqrt(self, n_feature):
        #См. выше в отладке
        return __get_feature_ids_sqrt(n_feature)
    
    def __get_feature_ids_log2(self, n_feature):
        #См. выше в отладке
        return __get_feature_ids_log2(n_feature)

    def __get_feature_ids_N(self, n_feature):
        #См. выше в отладке
        return __get_feature_ids_N(n_feature)
    
    def __sort_samples(self, x, y):
        sorted_idx = x.argsort()
        return x[sorted_idx], y[sorted_idx]

    def __div_samples(self, x, y, feature_id, threshold):
        left_mask = x[:, feature_id] > threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]

    def __find_threshold(self, x, y):
        # см. выше
        sorted_x, sorted_y = self.__sort_samples(x, y)
        class_number = np.unique(y).shape[0]
        
        #  см. выше
        splitted_sorted_y = sorted_y[self.min_samples_split:-self.min_samples_split]
        r_border_ids = np.where(splitted_sorted_y[:-1] != splitted_sorted_y[1:])[0] + (self.min_samples_split + 1)
        
        if len(r_border_ids) == 0:
            return float('+inf'), None
        
        #  см. выше
        eq_el_count = r_border_ids - np.append([self.min_samples_split], r_border_ids[:-1])
        one_hot_code = np.zeros((r_border_ids.shape[0], class_number))
        one_hot_code[np.arange(r_border_ids.shape[0]), sorted_y[r_border_ids - 1]] = 1
        class_increments = one_hot_code * eq_el_count.reshape(-1, 1)
        class_increments[0] = class_increments[0] + np.bincount(sorted_y[:self.min_samples_split], minlength=class_number)
        
        #  см. выше
        l_class_count = np.cumsum(class_increments, axis=0)        
        r_class_count = np.bincount(y) - l_class_count
        l_sizes = r_border_ids.reshape(l_class_count.shape[0], 1)
        r_sizes = sorted_y.shape[0] - l_sizes

        
        #как-то считаем критерий информативности, при этом возвращаем массив [1,2,34]
        gs = self.G_function(l_class_count, l_sizes, r_class_count, r_sizes)
        #находим минимальное значение
        idx = np.argmin(gs)
    
        # значение границы
        left_el_id = l_sizes[idx][0]
        #возвращаем минимальное значение информативности и значение границы
        return gs[idx], (sorted_x[left_el_id-1] + sorted_x[left_el_id]) / 2.0

    def __fit_node(self, x, y, node_id, depth, pred_f=-1):
        # Ваш код
        # Необходимо использовать следующее:
        # self.LEAF_TYPE
        # self.NON_LEAF_TYPE

        # self.tree
        # self.max_depth
        # self.sufficient_share
        # self.min_samples_split

        # self.get_feature_ids
        # self.__find_threshold
        # self.__div_samples
        # self.__fit_node
        
        #придется делать через рекурсию, так как такой интерфейс, хотя для python не очень хорошо
        #сначала определим что мы НЕ в LEAF:
        type_leaf = self.NON_LEAF_TYPE
        if depth >= self.max_depth:
            type_leaf = self.LEAF_TYPE
        
        share = np.max(np.unique(y, return_counts=True)[1])/y.shape[0]
        if share >= self.sufficient_share:
            type_leaf = self.LEAF_TYPE
        if y.shape[0] <= self.min_samples_split:
             type_leaf = self.LEAF_TYPE
        
        if type_leaf == self.LEAF_TYPE:
            self.__process_leaf_node(y, node_id)
            return
        
        
    def __process_leaf_node(y, node_id):
        clas, count = np.unique(y, return_counts=True)
        max_class = clas[np.argmax(count)]
        probability = np.max(count) / y.shape[0]
        self.tree[node_id] = [self.LEAF_TYPE, max_class , probability]
        
    def fit(self, x, y):
        self.num_class = np.unique(y).size
        self.__fit_node(x, y, 0, 0) 

    def __predict_class(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]

    def __predict_probs(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_probs(x, 2 * node_id + 1)
            else:
                return self.__predict_probs(x, 2 * node_id + 2)
        else:
            return node[2]
        
    def predict(self, X):
        return np.array([self.__predict_class(x, 0) for x in X])
    
    def predict_probs(self, X):
        return np.array([self.__predict_probs(x, 0) for x in X])

    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)

In [4]:
df = pd.read_csv('./cs-training.csv', sep=',').dropna()
df.shape

(120269, 11)

In [ ]:
x = df.as_matrix(columns=df.columns[1:])
y = df.as_matrix(columns=df.columns[:1])
y = y.reshape(y.shape[0])

In [ ]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

## Проверка скорости работы

In [ ]:
t1 = time()
my_clf.fit(x, y)
t2 = time()
print(t2 - t1)

t1 = time()
clf.fit(x, y)
t2 = time()
print(t2 - t1)

## Проверка качества работы

In [ ]:
gkf = KFold(n_splits=5, shuffle=True)

In [ ]:
for train, test in gkf.split(x, y):
    X_train, y_train = x[train], y[train]
    X_test, y_test = x[test], y[test]
    my_clf.fit(X_train, y_train)
    print(accuracy_score(y_pred=clf.predict(X_test), y_true=y_test))

In [ ]:
for train, test in gkf.split(x, y):
    X_train, y_train = x[train], y[train]
    X_test, y_test = x[test], y[test]
    clf.fit(X_train, y_train)
    print(accuracy_score(y_pred=clf.predict(X_test), y_true=y_test))

# Применить для задачи Titanic 